# How To Issue Natural Language Queries To An Existing Pinecone Index via LlamaIndex

A few weeks ago, I had the privilege of giving a talk & judging the finalists at [AGI House](https://agihouse.ai/)'s [Build / Break / Test](https://partiful.com/e/QKjcXJOCb2IHiuAVX9L6) hackathon.

Multiple engineers came up to me asking for a concise way of querying an *existing* Pinecone index via LlamaIndex, especially one that was [namespace-ed](https://docs.pinecone.io/docs/namespaces). While it's not currently possible to use [LlamaIndex's popular `as_query_engine()` method](https://gpt-index.readthedocs.io/en/latest/core_modules/query_modules/query_engine/root.html) to query an existing Pinecone index with namespaces, there is still a beautiful way to get the job done:

In [2]:
# Make sure you have the libraries below installed

import os

from llama_index import VectorStoreIndex
from llama_index import ServiceContext, OpenAIEmbedding
from llama_index.indices.vector_store import VectorIndexRetriever
from llama_index.llms import OpenAI
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.vector_stores import PineconeVectorStore

In [15]:
# You'll need to set your Pinecone and OpenAI environment variables.
# You can find your Pinecone API key and environment name in the UI at app.pinecone.io, once you've made an account.

os.environ['PINECONE_ENVIRONMENT'] = '<get at app.pinecone.io>'
os.environ['PINECONE_API_KEY'] = '<get at app.pinecone.io>'
os.environ['OPENAI_API_KEY'] = '<get from OpenAI>'

In [16]:
# Initialize your Pinecone vector store, passing in your environment variables.
# Here, you will input the name of your exiting Pinecone index as `index_name`
# If your index has a namespace, you'll put it in the `namespace` parameter
# And if your metadata dicts (every vector has an optional metadata object you can populate) have a diff key than the default of `text`,
    # you can put that in the `text_key` parameter.

vector_store = PineconeVectorStore(api_key=os.getenv('PINECONE_API_KEY'),
                                   environment=os.getenv('PINECONE_ENVIRONMENT'),
                                   index_name='<your existing index\'s name>',
                                   namespace="<optional namespace>",
                                   text_key='<optional override of key `text` in case your index has a diff key in the metadata>')

In [17]:
# Build your Service Context object
# You need the `llm` object to generate text
# Your need the `embed_model` to vectorize your query. Note: this has to be the same model used to create the vectors in your index.

llm = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))  # Default is gpt-3.5-turbo
embed_model = OpenAIEmbedding(api_key=os.getenv('OPENAI_API_KEY'))  # Default is text-embedding-ada-002
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)

In [18]:
# Build your Index object

index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [19]:
# Build your Retriever object

vector_index_retriever = VectorIndexRetriever(index)

In [20]:
# Build your Query Engine (this is where you might use the `as_query_engine()` method in other use cases)

query_engine = RetrieverQueryEngine(vector_index_retriever)

In [21]:
# Query your engine!

response = query_engine.query("What is a dog?")

In [22]:
response

Response(response='A dog is a domesticated mammal that is commonly kept as a pet or used for various purposes such as hunting, herding, and guarding. Dogs are known for their loyalty, companionship, and ability to be trained. They come in various breeds, sizes, and colors, and are known for their keen sense of smell and hearing.', source_nodes=[NodeWithScore(node=TextNode(id_='b5d400da-cebd-473b-a5c8-f018cbd038da', embedding=None, metadata={'chunk_num': 100.0, 'title': 'Albedo', 'url': 'https://en.wikipedia.org/wiki/Albedo'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='deafcfc45a319cfd306a263d792cb67e12957bcb48c62b7d696f0574729ad422', text='Clouds', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.803958595), NodeWithScore(node=TextNode(id_='df870970-c9f6-4497-8133-682d2affa056', embedding=None, metadata={'chunk_num': 254.0, 'title': 'Aut

## Give us a shout on [Twitter](https://twitter.com/pinecone) or [LinkedIn](https://www.linkedin.com/company/pinecone-io/) with any follow-up questions :) We would love to hear from you